ORM(Object Relation Mapping)

Prigramming technique for converting data 

In [35]:
conda install -c anaconda sqlalchemy


Note: you may need to restart the kernel to use updated packages.


In [2]:
import sqlalchemy

In [3]:
sqlalchemy.__version__

'1.3.1'

In [4]:
from sqlalchemy import create_engine

In [10]:
engine = create_engine("sqlite:///orm.db",echo=True)
# engine = create_engine("sqlite:///:memory",echo=True)
# engine = create_engine("sqlite://",echo=True)

In [8]:
print(engine)

Engine(sqlite:///orm.db)


In [9]:
sqlalchemy.create_engine??

sqlite로 바로 로컬에서 실행할 수 있다.

-> 현재까지 접근 준비

In [ ]:
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey

metadata = MetaData()
users = Table('users', metadata,
             Column('id',Integer, primary_key=True),
             Column('name',String),
             Column('fullname',String),
             )
addresses = Table('addresses', metadata,
             Column('id',Integer, primary_key=True),
             Column('user_id', None,ForeignKey('users.id')),
             Column('email_address',String, nullable=False),
             )

데이터형 String은 내부적으로 DB에 맞는 데이터타입으로 적절하게 바꿔준다.

In [15]:
users

Table('users', MetaData(bind=None), Column('id', Integer(), table=<users>, primary_key=True, nullable=False), Column('name', String(), table=<users>), Column('fullname', String(), table=<users>), schema=None)

In [16]:
metadata.tables

immutabledict({'users': Table('users', MetaData(bind=None), Column('id', Integer(), table=<users>, primary_key=True, nullable=False), Column('name', String(), table=<users>), Column('fullname', String(), table=<users>), schema=None), 'addresses': Table('addresses', MetaData(bind=None), Column('id', Integer(), table=<addresses>, primary_key=True, nullable=False), Column('user_id', Integer(), ForeignKey('users.id'), table=<addresses>), Column('email_address', String(), table=<addresses>, nullable=False), schema=None)})

아직 데이터베이스에 만들어지지 않음.

immutable -> 테이블 수정 불가

In [17]:
metadata.bind
# 연결 유무 확인

In [18]:
metadata.create_all(engine)

2019-07-09 11:18:36,859 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-07-09 11:18:36,860 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 11:18:36,861 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-07-09 11:18:36,862 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 11:18:36,865 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2019-07-09 11:18:36,866 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 11:18:36,867 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("addresses")
2019-07-09 11:18:36,868 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 11:18:36,870 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2019-07-09 11:18:36,870 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 11:18:36,879 INFO sqlalchemy.engine.base.Engine COMMIT
2019-07-09 11:18:36,881 INFO sqlalchemy.engine.b

In [19]:
insert = users.insert()
print(insert)

insert = users.insert().values(
    name='kim', fullname='Anonymous, kim'
)
print(insert)

insert.compile().params

INSERT INTO users (id, name, fullname) VALUES (:id, :name, :fullname)
INSERT INTO users (name, fullname) VALUES (:name, :fullname)


{'name': 'kim', 'fullname': 'Anonymous, kim'}

In [24]:
engine.connect??

In [25]:
conn = engine.connect() 

In [26]:
conn

In [27]:
insert.bind = engine

In [28]:
str(insert)

'INSERT INTO users (name, fullname) VALUES (?, ?)'

In [29]:
cur = conn.execute(insert)

2019-07-09 11:34:33,718 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname) VALUES (?, ?)
2019-07-09 11:34:33,719 INFO sqlalchemy.engine.base.Engine ('kim', 'Anonymous, kim')
2019-07-09 11:34:33,722 INFO sqlalchemy.engine.base.Engine COMMIT


In [30]:
cur.inserted_primary_key

[1]

In [31]:
insert = users.insert()

In [32]:
result = conn.execute(
    insert,
    name="lee",
    fullname="Unknown, Lee"
)

2019-07-09 11:35:58,357 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname) VALUES (?, ?)
2019-07-09 11:35:58,358 INFO sqlalchemy.engine.base.Engine ('lee', 'Unknown, Lee')
2019-07-09 11:35:58,362 INFO sqlalchemy.engine.base.Engine COMMIT


In [33]:
result.inserted_primary_key

[2]

In [38]:
conn.execute(addresses.insert(),[
    {"user_id":1,"email_address":"anonymous.kim@test.com"},
    {"user_id":2,"email_address":"unknow.kim@test.com"},
])

2019-07-09 11:45:43,475 INFO sqlalchemy.engine.base.Engine INSERT INTO addresses (user_id, email_address) VALUES (?, ?)
2019-07-09 11:45:43,476 INFO sqlalchemy.engine.base.Engine ((1, 'anonymous.kim@test.com'), (2, 'unknow.kim@test.com'))
2019-07-09 11:45:43,479 INFO sqlalchemy.engine.base.Engine COMMIT


In [36]:
users

Table('users', MetaData(bind=None), Column('id', Integer(), table=<users>, primary_key=True, nullable=False), Column('name', String(), table=<users>), Column('fullname', String(), table=<users>), schema=None)

In [37]:
from sqlalchemy.sql import select

In [39]:
cur = conn.execute(select([users])) 

2019-07-09 11:45:47,786 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users
2019-07-09 11:45:47,787 INFO sqlalchemy.engine.base.Engine ()


In [40]:
cur.fetchall()

[(1, 'kim', 'Anonymous, kim'), (2, 'lee', 'Unknown, Lee')]

#### Conjections

In [41]:
from sqlalchemy import and_,or_,not_
print(users.c.id == addresses.c.user_id)
print(users.c.id ==1 )

users.id = addresses.user_id
users.id = :id_1


파이썬 연산자를 사용할 수있다.

In [42]:
result = conn.execute(select([users]).where(users.c.id ==1))
for row in result:
    print(row)

2019-07-09 13:17:14,204 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname 
FROM users 
WHERE users.id = ?
2019-07-09 13:17:14,205 INFO sqlalchemy.engine.base.Engine (1,)
(1, 'kim', 'Anonymous, kim')


In [44]:
result = conn.execute(
    select([users, addresses]).\
    where(users.c.id==addresses.c.user_id)
)
for row in result:
    print(row)


2019-07-09 13:20:28,481 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.name, users.fullname, addresses.id, addresses.user_id, addresses.email_address 
FROM users, addresses 
WHERE users.id = addresses.user_id
2019-07-09 13:20:28,482 INFO sqlalchemy.engine.base.Engine ()
(1, 'kim', 'Anonymous, kim', 1, 1, 'anonymous.kim@test.com')
(2, 'lee', 'Unknown, Lee', 2, 2, 'unknow.kim@test.com')
(1, 'kim', 'Anonymous, kim', 3, 1, 'anonymous.kim@test.com')
(2, 'lee', 'Unknown, Lee', 4, 2, 'unknow.kim@test.com')


In [45]:
from sqlalchemy import join

In [48]:
print(users.join(addresses))
# pk와 fk 설정을 해서 알아서 where on절이 구성된다. 

users JOIN addresses ON users.id = addresses.user_id


In [49]:
print(users.join(addresses,users.c.id == addresses.c.user_id))

users JOIN addresses ON users.id = addresses.user_id


In [52]:
query = select([
    users.c.id,
    users.c.fullname,
    addresses.c.email_address
    ]).select_from(
    users.join(addresses)
    )

In [53]:
result = conn.execute(query).fetchall()
for row in result:
    print(row)

2019-07-09 13:27:49,697 INFO sqlalchemy.engine.base.Engine SELECT users.id, users.fullname, addresses.email_address 
FROM users JOIN addresses ON users.id = addresses.user_id
2019-07-09 13:27:49,698 INFO sqlalchemy.engine.base.Engine ()
(1, 'Anonymous, kim', 'anonymous.kim@test.com')
(2, 'Unknown, Lee', 'unknow.kim@test.com')
(1, 'Anonymous, kim', 'anonymous.kim@test.com')
(2, 'Unknown, Lee', 'unknow.kim@test.com')


In [55]:
metadata.sorted_tables

[Table('users', MetaData(bind=None), Column('id', Integer(), table=<users>, primary_key=True, nullable=False), Column('name', String(), table=<users>), Column('fullname', String(), table=<users>), schema=None),
 Table('addresses', MetaData(bind=None), Column('id', Integer(), table=<addresses>, primary_key=True, nullable=False), Column('user_id', Integer(), ForeignKey('users.id'), table=<addresses>), Column('email_address', String(), table=<addresses>, nullable=False), schema=None)]

In [60]:
metadata.clear()
# metadata 날리기

In [59]:
metadata.tables

immutabledict({})